In [498]:
import numpy as np
import csv
%matplotlib inline
import pandas as pd
from frgpascal.hardware.sampletray import SampleTray, AVAILABLE_VERSIONS as sampletray_versions
from frgpascal.hardware.liquidlabware import LiquidLabware, AVAILABLE_VERSIONS as liquid_labware_versions
from frgpascal.hardware.hotplate import AVAILABLE_VERSIONS as hotplate_versions
from frgpascal.experimentaldesign.helpers import build_sample_list, plot_tray, handle_liquids, samples_to_dataframe
from frgpascal.experimentaldesign.tasks import *
from frgpascal.experimentaldesign.scheduler import Scheduler

from scipy.optimize import nnls
import itertools as itt

In [499]:
#### name parsing helper functions
def components_to_name(components, delimiter="_"):
    composition_label = ""
    for c, n in components.items():
        if n > 0:
            composition_label += "{0}{1:.2f}{2}".format(c, n, delimiter)

    return composition_label[:-1]


def name_to_components(
    name,
    factor=1,
    delimiter="_",
):
    """
    given a chemical formula, returns dictionary with individual components/amounts
    expected name format = 'MA0.5_FA0.5_Pb1_I2_Br1'.
    would return dictionary with keys ['MA, FA', 'Pb', 'I', 'Br'] and values [0.5,.05,1,2,1]*factor
    """
    components = {}
    for part in name.split(delimiter):
        species = part
        count = 1.0
        for l in range(len(part), 0, -1):
            try:
                count = float(part[-l:])
                species = part[:-l]
                break
            except:
                pass
        if species == "":
            continue
        components[species] = count * factor
    return components


In [500]:
def solutions_to_matrix(solutions:list):
    if isinstance(solutions, Solution):
        solutions = [solutions]

    # get possible solution components from stock list
    components = set()
    for s in solutions:
        components.update(s.solute_dict.keys(), s.solvent_dict.keys())
    components = list(
        components
    )  # sets are not order-preserving, lists are - just safer this way

    # organize components into a stock matrix, keep track of which rows are solvents
    stock_matrix = np.zeros((len(components), len(solutions)))
    solvent_idx = set()
    for n, s in enumerate(solutions):
        for m, c in enumerate(components):
            if c in s.solute_dict:
                stock_matrix[m, n] = s.solute_dict[c] * s.molarity
            elif c in s.solvent_dict:
                stock_matrix[m, n] = s.solvent_dict[c]
                solvent_idx.add(m)
    solvent_idx = list(solvent_idx)

    return stock_matrix
    

In [10]:
target_solutions = [
    Solution(
        solutes='MA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='FA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='FA0.25_MA0.75_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='FA0.75_MA0.25_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='MA0.5_FA0.5_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
]


In [11]:
stock_solutions = [
    Solution(
        solutes='MA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solutes='FA_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    ),
    Solution(
        solvent='DMF',
    ),
    Solution(
        solvent='DMSO',
    ),
    Solution(
        solvent='Chlorobenzene',
    ),
    Solution(
        solvent='MethylAcetate',
    ),
    Solution(
        solutes='spiro',
        solvent='IPA',
        molarity=1
    ),
]

In [12]:
target = Solution(
        solutes='MA0.2_FA0.8_Pb_I3',
        solvent='DMF9_DMSO1',
        molarity=1
    )

In [510]:
class SolutionMixer:
    def __init__(self, stock_solutions:list, targets:dict={}):
        self.target_solutions = list(targets.keys())
        self.target_volumes = targets #dict of solution:volume
        self.solutions = stock_solutions + self.target_solutions
        self.solution_matrix, self.solvent_idx, self.components = self._solutions_to_matrix(stock_solutions + self.target_solutions)
        self.stock_idx = [i for i in range(len(stock_solutions))] #rows of solution matrix that belong to stock solutions

    def _solutions_to_matrix(self, solutions:list, components:list = None):
        """Converts a list of solutions into a matrix of molarity of each component in each solution

        Args:
            solutions (list): Solution objects
            components (list, optional): individual components that cover all solutions. If none, will generate this from the solution list

        Returns:
            solution_matrix, solvent_idx, components

            solution_matrix = rows are solutions, columns are components

        """
        if isinstance(solutions, Solution):
            solutions = [solutions]

        # get possible solution components from stock list
        if components is None:
            components = set()
            for s in solutions:
                components.update(s.solute_dict.keys(), s.solvent_dict.keys())
            components = list(
                components
            )  # sets are not order-preserving, lists are - just safer this way

        # organize components into a stock matrix, keep track of which rows are solvents
        solution_matrix = np.zeros((len(solutions), len(components)))
        solvent_idx = set()
        for m, s in enumerate(solutions):
            for n, c in enumerate(components):
                if c in s.solute_dict:
                    solution_matrix[m, n] = s.solute_dict[c] * s.molarity
                elif c in s.solvent_dict:
                    solution_matrix[m, n] = s.solvent_dict[c]
                    solvent_idx.add(m)
        solvent_idx = list(solvent_idx)
        
        return solution_matrix, solvent_idx, components

    def _solution_to_vector(self, target:Solution, volume=1):
        # organize target solution into a matrix of total mols desired of each component
        target_matrix = np.zeros((len(self.components),))
        for m, c in enumerate(self.components):
            if c in target.solute_dict:
                target_matrix[m] = target.solute_dict[c] * target.molarity * volume
            elif c in target.solvent_dict:
                target_matrix[m] = target.solvent_dict[c] * volume
        return target_matrix.T

    def _calculate_mix(self, target, solution_indices:list=None, tolerance=1e-3, min_fraction=0):
        if solution_indices is None:
            solution_indices = list(range(len(self.solutions)))
        A = self.solution_matrix[solution_indices]
        b = self._solution_to_vector(target)
        x, err = nnls(A.T, b, maxiter=1e3)
        x[x<1e-10] = 0
        if err > tolerance:
            return np.nan
        if np.logical_and(x > 0, x<min_fraction).any():
            return np.nan
        
        x_full = np.zeros((len(self.solutions),))
        for idx, x_ in zip(solution_indices, x):
            x_full[idx] = x_
        return x_full

    def mix(self, target, volume, solution_indices = None, tolerance=1e-3, min_volume=0, verbose=False):
        min_fraction = min_volume / volume
        if solution_indices is None or solution_indices == 'stock':
            solution_indices = self.stock_idx
        elif solution_indices == 'all':
            solution_indices = list(range(len(self.solutions)))

        possible_mixtures = []
        for i in range(1,len(self.solutions)):
            for idx in itt.combinations(solution_indices,i):
                x = self._calculate_mix(target, solution_indices=list(idx), tolerance=tolerance, min_fraction=min_fraction)
                if not np.isnan(x).any():
                    possible_mixtures.append(x)
        if len(possible_mixtures) == 0:
            return np.nan
        possible_mixtures.sort(key = lambda x: -min(x[x>0])) #sort such that the mixture with the largest minimum fraction is first
        if verbose:
            return possible_mixtures*volume
        return possible_mixtures[0]*volume

    def mixture_graph(self, min_volume=20):
        self.graph = np.zeros((len(self.solutions), len(self.solutions)))
        self.availableidx = self.stock_idx.copy()
        # self.graph[np.diag_indices(len(self.solutions))] = 1
        attempts = 0
        while len(self.availableidx) < len(self.solutions):
            if attempts > 5:
                print('Could not find a solution') 
                break
            for i in range(len(self.solutions)):
                if i in self.availableidx:
                    continue
                x = self.mix(
                    target=self.solutions[i],
                    volume = self.target_volumes[self.solutions[i]],
                    solution_indices=self.availableidx,
                    min_volume=min_volume)
                if not np.isnan(x).any():
                    self.graph[i,:] = x
                    self.availableidx.append(i)
            attempts += 1
        # return self.graph
    







In [511]:
densetargets = []
for i in np.linspace(0.1, 0.5, 10):
    densetargets.append(Solution(
        solutes=f"MA{i:.2f}_FA{1-i:.2f}_Pb_I3",
        solvent="DMF9_DMSO1",
        molarity=1
    ))

In [512]:
sm = SolutionMixer(stock_solutions, {t:60 for t in densetargets})

In [573]:
sm.mixture_graph(min_volume=25)

In [587]:
edges = [
    (
        sm.solutions[m],
        sm.solutions[n],
        sm.graph[m,n]
    ) for m,n in np.ndindex(sm.graph.shape)
    if sm.graph[m,n] > 0
    ]
g = nx.DiGraph()
g.add_weighted_edges_from(edges)

In [588]:
nodelist = [n for n in g.nodes]

In [589]:
gmat = nx.convert_matrix.to_numpy_matrix(g, nodelist=nodelist)

In [592]:
gmat.sum(axis=0)

matrix([[ 30.        , 182.71304348,  56.08695652,   0.        ,
          90.        ,   0.        ,  86.66666667,  33.6       ,
           0.        ,  30.        ,  33.33333333,  57.6       ]])

In [590]:
gmat[gmat>0].min()

26.086956521739143

In [579]:
[gen for gen in nx.topological_generations(g)][::-1]


[[0, 1], [16], [11, 15], [10, 13], [7, 14], [8, 9, 12]]

[[<Solution>1M MA0.10_FA0.90_Pb_I3 in DMF9_DMSO1,
  <Solution>1M MA0.14_FA0.86_Pb_I3 in DMF9_DMSO1,
  <Solution>1M MA0.28_FA0.72_Pb_I3 in DMF9_DMSO1,
  <Solution>1M MA0.32_FA0.68_Pb_I3 in DMF9_DMSO1,
  <Solution>1M MA0.50_FA0.50_Pb_I3 in DMF9_DMSO1],
 [<Solution>1M MA0.19_FA0.81_Pb_I3 in DMF9_DMSO1,
  <Solution>1M MA0.23_FA0.77_Pb_I3 in DMF9_DMSO1,
  <Solution>1M MA0.41_FA0.59_Pb_I3 in DMF9_DMSO1],
 [<Solution>1M MA0.37_FA0.63_Pb_I3 in DMF9_DMSO1,
  <Solution>1M MA0.46_FA0.54_Pb_I3 in DMF9_DMSO1],
 [<Solution>1M MA_Pb_I3 in DMF9_DMSO1, <Solution>1M FA_Pb_I3 in DMF9_DMSO1]]

In [521]:
next(gen)

StopIteration: 